Sources: 
- https://medium.com/@bdov_/https-medium-com-bdov-python-this-is-an-object-that-is-an-object-everything-is-an-object-fff50429cd4b
- https://medium.com/@bdov_/https-medium-com-bdov-python-objects-part-ii-demystifying-cpython-shared-objects-fce1ec86dd63
- https://medium.com/@bdov_/https-medium-com-bdov-python-objects-part-iii-string-interning-625d3c7319de
- https://medium.com/@bdov_/python-objects-part-iv-first-class-everything-7da3945e3552
- GPT

In Python, *everything* is an object.
  
> One of my goals for Python was to make it so that all objects were “first class.” By this, I meant that I wanted all objects that could be named in the language (e.g., integers, strings, functions, classes, modules, methods, etc.) to have equal status. That is, they can be assigned to variables, placed in lists, stored in dictionaries, passed as arguments, and so forth. — Guido Van Rossum

Every object has an *identity*, *value*, and *type*: 
- **Identity**: never changes after the object is created (it's similar to the object's memory address).  
  - The `is` operator compares object identities, and the `id()` function returns an integer representing the object's identity (in [CPython](https://pt.wikipedia.org/wiki/CPython), it returns the memory address of the object).  
- **Type**: determines the operations an object supports and defines all possible values for that object.  
    - Python’s type system: *Duck Typing* (“If it looks like a duck and quacks like a duck, it’s a duck”).  
        - With **nominal typing**, an object belongs to a given type only if it is declared as such (or through mechanisms like class inheritance). With **duck typing**, an object is considered to be of a given type if it has all the methods and properties required by that type — therefore, no explicit type checking is needed.  
    - Type is immutable.
    - The `type()` function returns the object’s type, which is itself an object.  
- The **value** of an object (how it "stores") may change.
    - While `is` checks identity (are both variables pointing to the exact same object in memory?), `==` checks equality (do the objects have the same value?)

According to the [source code of CPython](https://github.com/python/cpython/blob/main/Include/object.h):

> Objects are [C] structures allocated on the heap [...] Objects are never allocated statically or on the stack (Type objects are exceptions to the first rule; the standard types are represented by statically initialized type objects).
> 
> An object has a 'reference count' that is increased or decreased when a pointer to the object is copied or deleted; when the reference count reaches zero there are no references to the object left and it can be removed from the heap. An object has a 'type' that determines what it represents and what kind of data it contains.  An object's type is fixed when it is created. Types themselves are represented as objects; an object contains a pointer to the corresponding type object.  The type itself has a type pointer pointing to itself!
>
> Objects do not float around in memory; once allocated an object keeps the same size and address.  Objects that must hold variable-size data can contain pointers to variable-size parts of the object.  Not all objects of the same type have the same size; but the size cannot change after allocation.  (These restrictions are made so a reference to an object can be simply a pointer -- moving an object would require updating all the pointers, and changing an object's size would require moving it if there was another object right next to it.)
>
> Objects are always accessed through pointers of the type 'PyObject *'. The type 'PyObject' is a structure that only contains the reference count and the type pointer. The actual memory allocated for an object contains other data that can only be accessed after casting the pointer to a pointer to a longer structure type.  This longer type must start with the reference count and type fields.
>
> A standard interface exists for objects that contain an array of items whose size is determined when the object is allocated.
<figure>
    <img src="imgs/PyObject_structure.webp" alt="PyObject C struct" width=680 text>
    </br>
    <img src="imgs/PyVarObject_structure.png" alt="PyVarObject C struct", width=530>
    <figcaption>CPython source code</figcaption>
</figure>
</br>
<figure>
    <img src="imgs/c_memory_layout.jpg" alt="Mem layout", width=580>
    <figcaption>Just a reminder: basic memory layout</figcaption>
</figure>

In [34]:
x = 1
type(x), id(x)

(int, 4314816512)

In [35]:
y = 2
type(y), id(y)

(int, 4314816544)

In [36]:
x is y # Checks if both variables point to the same object in memory

False

In [ ]:
x == y # Checks if both variables have the same value

True

In [ ]:
def function():
    return "Hello, World!"

type(function), id(function) # function is an object too

(function, 4364076832)

In [ ]:
class Class:
    pass

type(Class), id(Class) # Class is also an object

(type, 4354191984)

**Immutable objects** are objects whose value cannot change after creation (ex.: `int`, `float`, `bool`, `str`, and `tuple`). For those objects, after being instructed to create an instance of an object, Python first checks its existing memory. If an instance of that object already exists, instead of allocating memory to create a new one, it will use the pre-existing value. However, immutability alone does not determine whether Python reuses an existing object or creates a new one: object reuse is an **implementation optimization specific to CPython**, not a language guarantee.

---

CPython pre-instantiates and stores **specific instances and ranges of certain immutable types**. These are known as **shared objects**. Internally, they are "global" objects that can be reused by all code running within the same interpreter process.

The motivation behind this design is performance. Some values appear extremely frequently in programs, and repeatedly allocating memory for them would be wasteful. When such a cached value is needed, Python can simply return a reference to the existing object instead of creating a new one.

---

**Integers**

For integers, CPython preloads and shares all values in the range **-5 to 256 (inclusive)**. Any reference to an integer within this range typically points to the same object in memory. Integers outside this range are usually instantiated as new objects, even if they have the same value, unless compiler optimizations apply.

---

**Strings**

Strings behave similarly to integers, but with **more nuanced rules**.

CPython uses **string interning**:
- Short string literals and strings that resemble identifiers (letters, digits, underscores) are often interned automatically.
- Strings containing spaces, punctuation, or those created dynamically at runtime are **not reliably interned**.
- Explicit interning can be requested using `sys.intern()`.

As a result, two strings with the same value may or may not refer to the same object, depending on how and where they are created.

---

**Tuples**

Although tuples are immutable, their instantiation behavior differs from that of integers and strings.

The key reason is that **tuples can contain mutable objects**. While the tuple itself cannot change, the objects it references might. Because of this, CPython generally treats tuple creation as a **fresh allocation**, similar to mutable objects (a notable exception is the empty tuple `()`).

---

![Python Object Instantiation Chart](imgs/python_obj_instantiaction.webp)

In [32]:
x = 1
print(id(x))
x += 1
print(id(x)) # id changes because integers are immutable, so a new object is created

print("")

x = "hello"
print(id(x))
x += " world"
print(id(x))  # id changes because strings are immutable, so a new object is created

4314816512
4314816544

4358836688
4364303984


In [48]:
t = (1, 2, 3)
z = (1, 2, 3)
print(t is z)  # False, because tuples are immutable and two different tuple objects are created
print(t == z)  # True, because the contents of both tuples are the same

False
True


In [23]:
a = 256
b = 256
c = 257
d = 257
print(a is b)  # True, because small integers are cached by Python
print(c is d)  # False, because integers larger than 256 are not cached by Python

True
False


In [22]:
a = -5
b = -5
c = -6
d = -6
print(a is b)  # True, because small integers are cached by Python
print(c is d)  # False, because integers larger than 256 are not cached by Python

True
False


In [31]:
a = (1, 2, 3)
try:
    a[0] = 0  # This will raise an error because tuples are immutable
except TypeError as e:
    print(e)

'tuple' object does not support item assignment


Strings behave similarly to integers, but with more rules and more traps.

In [2]:
a = "hello"
b = "hello"
c = "hello world!"
d = "hello world!"
print(a is b)  # True, because short strings are interned by Python
print(c is d)  # False, because longer strings are not interned by Python

try:
    c[0] = "H"  # This will raise an error because strings are immutable
except TypeError as e:
    print(e)

True
False
'str' object does not support item assignment


**Mutable objects** are those whose value can change (list and dict are examples of mutable objects). Different from immutable, they are always instantiated immediately: each mutable object must have its own unique identity.

In [29]:
l = [1, 2, 3]
print(id(l))
l[0] = 0
l.append(4)
print(id(l))  # id remains the same because lists are mutable

4364060480
4364060480


In [27]:
l = [1, 2, 3]
m = [1, 2, 3]
print(l is m)  # False, because lists are mutable and two different list objects are created
print(l == m)  # True, because the contents of both lists are the same

False
True


All the information about an object, including its value and methods, are referred to as the **attributes of a class**.

`dir()` is the built-in function to get a list of all the attributes of an object.

In [55]:
n = 1
print(len(dir(x)))
dir(x)

30


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'name']

To create our own types (custom classes) we use the keyword `class`.

In [61]:
class MyClass:
    pass

x = MyClass()
print(len(dir(x)))

29


In [62]:
x.name = "Some Name"
print(len(dir(x))) # dir now includes 'name' attribute
dir(x)

30


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'name']

Although the list `dir()` returned for `x` is similar to the returned for the integer `n`, they are not the same. The `__dict__` attribute is unique to custom classes. It is a dictionary used to store all arbitrary attributes of a user-defined class.

In [63]:
"__dict__" in dir(x), "__dict__" in dir(n)

(True, False)

As attributes are added to or removed from an instance of a custom class, they are added and removed from that instances’ `__dict__`.

In [66]:
print(x.__dict__)
x.test = 123
print(x.__dict__)

{'name': 'Some Name'}
{'name': 'Some Name', 'test': 123}


In [67]:
try:
    n.test = 123  # This will raise an error because integers do not have a __dict__
except AttributeError as e:
    print(e)

'int' object has no attribute 'test' and no __dict__ for setting new attributes


As mentioned, in Python, everything is an object — including classes: a class is itself an instance of the built-in `type` class, which is the class of classes (including itself).

`type` is the default metaclass (metaclass controls how Python builds a class internally, how Python constructs the MRO (Method Resolution Order), how multiple inheritance works, etc).

Classes in Python are created at runtime: an object representing **a class** (the *class object*) is created **at the moment the class statement is executed by Python**, not when you instantiate the class. 

Classes are callable objects (they implement the `__call__()` method) that act as factories: they create and return an instance of themselves.

In [70]:
type(MyClass)

type

In [71]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__static_attributes__': (),
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

Steps for classes and instances creation:

1. **Class creation**
    
    ``` python
    class A(B):
        x = 10
        def __init__(self, v):
            self.v = v
    ```
    
    1. Namespace creation:
        
        ```python
        {"x": 10, "__init__": <function ...>}
        ```
        
    2. Python collects the class name (`A`), the base classes (`B`), and the constructed namespace.
    
    3. It calls the metaclass (normally `type`):
        
        ```python
        A = type("A", (B,), {"x": 10, "__init__": ...})
        ```
        
        1. `type.__new__` creates the class object.
        2. `type.__init__` initializes the class object.
        3. Result: `A` is an object (an instance of `type`).
    
2. **Instance creation**

    ```python
    obj = A(5)
    ```
    
    1. `A(5)` triggers `A.__call__(5)`.
    2. Inside this method:
        
        1. Execution of `A.__new__`
            
            ```python
            instance = A.__new__(A, 5) # A must be passed because __new__ is static
            ```
            
           - Allocates the object on the heap and returns the newly created (but not yet initialized) instance.
           - If `__new__` returns an object of class `A` itself, then step 2.2 is executed.

        2. Execution of `A.__init__`
            
            ```python
            MyClass.__init__(instance, 5) # equivalent to instance.__init__(5)
            ```

In [75]:
class MyClass:
    def __new__(cls, *args, **kwds):
        print("Calling __new__ method")
        return super().__new__(cls)
    
    def __init__(self, *args, **kwds):
        print("Calling __init__ method")

x = MyClass()

Calling __new__ method
Calling __init__ method


`super()`: a function that allows accessing **superclass** methods without creating new instances. The **super** of a class is not always the “direct parent class” but rather **the next class in the MRO sequence.**

`object.__new__(cls, ...)` is a **special static method** responsible for creating a new instance of the class `cls`. It does not need to be declared with `@staticmethod`, and its first argument is always the class being instantiated. The remaining arguments are the same ones provided in the class call (for example, `MyClass(arg1, arg2)`), even though `__new__` does not initialize the instance. These arguments are passed so that `__new__` can decide how the object should be created (or whether it should be created at all) and are also useful for inspection, validation, or debugging before initialization occurs.


In [81]:
class MyClass:
    count = 0

    def __init__(self):
        self.name = "My Name"

x = MyClass()
print(x.__dict__)
print(MyClass.__dict__)
print(x.count)

{'name': 'My Name'}
{'__module__': '__main__', '__firstlineno__': 1, 'count': 0, '__init__': <function MyClass.__init__ at 0x1042cd4e0>, '__static_attributes__': ('name',), '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
0


The code above shows an important point: instance and class attributes represent, and are represented in, two separate namespaces. `count` didn't appear in `x` attributes as it was in the class-level namespace (`MyClass.__dict__`), but because of the *attribute lookup* mechanism (if the attribute is not in the instance, Python looks in the class and then in the parent classes) it could be accessed by the instance.

Note: the code bellow shows that it is not possible to modify a class attribute from the instance: if an assignment is made through the instance, Python creates (or overrides) an attribute only on the instance (a new field appears on the instance, completely independent of the class field).


In [84]:
x.count = 10
print(x.count)
print(MyClass.count)
print(x.__dict__)

10
0
{'name': 'My Name', 'count': 10}


**`object`** is the base class of **all instances** in Python, meaning that everything in Python is an object because all classes ultimately inherit from `object`, making it the last class in the method resolution order (MRO). **`type`** is the base class of **all classes** and serves as Python’s default metaclass. As part of Python’s object model, `object` is an instance of `type`, while `type` itself is a subclass of `object`. This circular relationship is intentional and forms a well-designed foundation of Python’s type system.


In [91]:
isinstance(MyClass, object), isinstance(MyClass, type)

(True, True)

In [92]:
isinstance(type, object), isinstance(object, type)

(True, True)

In [97]:
def say_hello(self):
    print("Hello, World! My name is", self.name)

MyClass = type("MyClass", (object,), {"say_hello": say_hello, "name": "Class Attribute"}) # Create a new class 'MyClass' with a method 'say_hello' and an attribute 'name'
print(MyClass.__dict__)  # Access the 'name' attribute

# Equivalent to:
# class MyClass:
#     name = "ExampleInstance"

#     def say_hello(self):
#         print("Hello, World! My name is", self.name)

{'say_hello': <function say_hello at 0x104850540>, 'name': 'Class Attribute', '__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


In [98]:
x = MyClass()  # Instantiate the class
x.say_hello()  # Call the method

Hello, World! My name is Class Attribute


**Singleton**

It's a design pattern whose goal is to ensure that only one instance of a class exists throughout the entire program — and that everyone who needs it receives exactly the same object.

The most common mechanism is overriding `__new__`:
    
```python
class Singleton:
    _inst = None

    def __new__(cls, *args, **kwargs):
        if cls._inst is None:
            cls._inst = super().__new__(cls)
        return cls._inst
```
    
`super().__new__` inside this class is calling the `__new__` of the highest base class in the MRO — which is `object`, since `Singleton` does not inherit from any class other than `object`.

In [100]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, value):
        self.value = value

singleton1 = Singleton(1)
print(singleton1.value)

singleton2 = Singleton(2)
print(singleton1.value)
print(singleton2.value)

1
2
2


In [101]:
print(singleton1 is singleton2)  # This should print True, indicating both are the same instance

True
